# AWS fine-tuning Resnet 18 for image classification using SageMaker

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.

This project is part of the AWS Machine Learning Engineer Nanodegree requisites. It consists in the fine-tuning of a pretrained big image classification model (Resnet 18) using SageMaker, the main AWS tool for machine learning developing and deploying. The project will re train the last fully connected layer of the model with a new set of images from https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip, that consists in dozens of different images of dog breeds, labeled with the corresponding dog's breed. Then we will test the accuracy of the model in recognizing different breeds.
In the process of getting this job done, we will perform a search of best hyperparameteres and monitorize the re-training of the model by using the Sagemaker hyperparameter tuner, profiler and debugger.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)


In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3
from sagemaker import get_execution_role

bucket= "project3dogsimagesclassification"## TODO: fill in
print("Default Bucket: {}".format(bucket))

import boto3
region = boto3.Session().region_name ## TODO: fill in## TODO: fill in
print("AWS Region: {}".format(region))

role = get_execution_role() ## TODO: fill in
print("RoleArn: {}".format(role))

Default Bucket: project3dogsimagesclassification
AWS Region: us-east-1
RoleArn: arn:aws:iam::187401717515:role/service-role/AmazonSageMaker-ExecutionRole-20220110T092400


## Dataset
In this cell we download the zip file containing the images ditributed in three subdirectories: train (train set) valid (validation set) test (test set)

Within those dirs there are sub folders with a number and a name of the dog's breed images it contain. Those directory names will serve as labels for our images.

The !(bang sign) means execute a command in console First, we download the file by using the wget linux command Second, we unzip the file by using the zip linux command

These folders with images go to newly created folders into the file system of the machine that SageMaker is using. Then, another command, aws performs the copy from the local machine file system to the s3 resource of our AWS account. Those files in S3 are ready to be consumed by our model, provided the rol we asigned to the SageMaker session has rights to read/write over our S3 resource.

In [ ]:
#TODO: Fetch and upload the data to AWS S3
#run once only
# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip

In [ ]:
!unzip dogImages.zip
;

In [37]:
#!ls dogImages/ | tail
"""
!mkdir dogImages/train/resguardo
!mv dogImages/train/00* dogImages/train/resguardo/
!mv dogImages/train/010* dogImages/train/resguardo/
!mv dogImages/train/011* dogImages/train/resguardo/
!mv dogImages/train/012* dogImages/train/resguardo/
!mv dogImages/train/013* dogImages/train/resguardo/
!mkdir dogImages/train/basura/
!mv dogImages/train/0* dogImages/train/basura/
!mv dogImages/train/1* dogImages/train/basura/
!rm -fr dogImages/train/basura/
!mv dogImages/train/resguardo/* dogImages/train/ 
!rmdir dogImages/train/resguardo/
!ls dogImages/train/ 

!mkdir dogImages/test/resguardo
!mv dogImages/test/00* dogImages/test/resguardo/
!mv dogImages/test/010* dogImages/test/resguardo/
!mv dogImages/test/011* dogImages/test/resguardo/
!mv dogImages/test/012* dogImages/test/resguardo/
!mv dogImages/test/013* dogImages/test/resguardo/
!mkdir dogImages/test/basura/
!mv dogImages/test/0* dogImages/test/basura/
!mv dogImages/test/1* dogImages/test/basura/
!rm -fr dogImages/test/basura/
!mv dogImages/test/resguardo/* dogImages/test/ 
!rmdir dogImages/test/resguardo/
!ls dogImages/test/ 


!mkdir dogImages/valid/resguardo
!mv dogImages/valid/00* dogImages/valid/resguardo/
!mv dogImages/valid/010* dogImages/valid/resguardo/
!mv dogImages/valid/011* dogImages/valid/resguardo/
!mv dogImages/valid/012* dogImages/valid/resguardo/
!mv dogImages/valid/013* dogImages/valid/resguardo/
!mkdir dogImages/valid/basura/
!mv dogImages/valid/0* dogImages/valid/basura/
!mv dogImages/valid/1* dogImages/valid/basura/
!rm -fr dogImages/valid/basura/
!mv dogImages/valid/resguardo/* dogImages/valid/ 
!rmdir dogImages/valid/resguardo/
!ls dogImages/valid/ 
"""

In [4]:
!ls dogImages/test/

001.Affenpinscher		    068.Flat-coated_retriever
002.Afghan_hound		    069.French_bulldog
003.Airedale_terrier		    070.German_pinscher
004.Akita			    071.German_shepherd_dog
005.Alaskan_malamute		    072.German_shorthaired_pointer
006.American_eskimo_dog		    073.German_wirehaired_pointer
007.American_foxhound		    074.Giant_schnauzer
008.American_staffordshire_terrier  075.Glen_of_imaal_terrier
009.American_water_spaniel	    076.Golden_retriever
010.Anatolian_shepherd_dog	    077.Gordon_setter
011.Australian_cattle_dog	    078.Great_dane
012.Australian_shepherd		    079.Great_pyrenees
013.Australian_terrier		    080.Greater_swiss_mountain_dog
014.Basenji			    081.Greyhound
015.Basset_hound		    082.Havanese
016.Beagle			    083.Ibizan_hound
017.Bearded_collie		    084.Icelandic_sheepdog
018.Beauceron			    085.Irish_red_and_white_setter
019.Bedlington_terrier		    086.Irish_setter
020.Belgian_malinois		    087.Irish_terrier
021.Belgian_sheepdog		    088.Irish_water_spaniel
022.Belg

In [65]:
import os
os.environ["DEFAULT_S3_BUCKET"] = bucket
!aws s3 sync ./dogImages/train s3://${DEFAULT_S3_BUCKET}/dogImages/train/
!aws s3 sync ./dogImages/test s3://${DEFAULT_S3_BUCKET}/dogImages/test/
!aws s3 sync ./dogImages/valid s3://${DEFAULT_S3_BUCKET}/dogImages/valid/
;

''

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [5]:
#TODO: Declare your HP ranges, metrics etc.
import os #for avoiding run the former cell
os.environ["DEFAULT_S3_BUCKET"] = bucket
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

"""
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    #The epsilon for adam and rmsprop. It is usually set to a small value to avoid division by 0 (around 1e-8).
    #"eps": ContinuousParameter(1e-9, 1e-7), #no lo vamos a usar. vamos a usar adam y no AdamW
    "batch_size": CategoricalParameter([16, 64, 256, 1024]),
    #"weight_decay": ContinuousParameter(1e-3, 1e-1),#option if using AdamW
    "epochs": IntegerParameter(2, 4)
    #"epochs": IntegerParameter(2, 8)
}
"""

#temporal:
#Best hyperparameters from hpo:
#lr: 0.0921110950826238
#batch_size: 256
#epochs: 3
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.092, 0.0924),
    "batch_size": CategoricalParameter([256, 257]),
    "epochs": IntegerParameter(3, 4)
}




objective_metric_name = "Accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": "Accuracy", "Regex": "Accuracy: ([0-9\\.]+)"}]

In [7]:
#TODO: Create estimators for your HPs
from sagemaker.pytorch import PyTorch

#estimator = # TODO: Your estimator here
estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    #sagemaker_session=session,
    #source_dir="source",
    instance_type="ml.m5.large"
)


#tuner = # TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
)



In [20]:
# TODO: Fit your HP Tuner
#tuner.fit() # TODO: Remember to include your data channels
#tuner.fit(wait=True)
#path = "s3://{}".format(bucket) #works with the reduced dataset (10% of original) while working on the notebook
path =  "s3://{}/dogImages".format(bucket) #we will use this when we have the complete dataset. 
#we tune only in the training set or val set too?
#tuner.fit({"train": path + '/train' }, wait=True)
print(path)
print(path + '/train')
tuner.fit({
'train': path + '/train',     
'valid': path + '/valid',  
'test': path + '/test'     
}, wait=True)
#tuner.fit({'train': path + '/train'}, wait=True)
#each line around 9min we wilc check using another kernel
#9min per line using ml.g4dn.xlarge
#using  ml.t3.medium start one line at 18:56:28, ends one line at 19:05:02 ttl 8:30min
#second line ended at 19:14:05, 9:03min
#third line ended at 19:23:00, 8:55min
#fourth line ended at 19:31:55, 8:55min
#fifth line ended at 19:40:55, 9:00min
#sixth line ended at 19:50:40, 8:45min
#seventh line never ended
#eighth line is shorter

#ml.t3.medium
#1st 8:55:00 9min
#2nd 9:04:00 9min
#3rd 9:13:00 9min
#4th 9:22:00 
#5th 9:31
#6th 9:40
#7th 9:55
#8th 10:04



#8:53 fin 9:55:05, 1:02hs 7 lines
#8:53 fin 10:03, 1:10hs 8 lines
#ttl 70min with ml.t3.medium


#started  15:10, ended 16:23 with smaller parameter range


s3://project3dogsimagesclassification/dogImages
s3://project3dogsimagesclassification/dogImages/train
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [8]:
# TODO: Get the best estimators and the best HPs

#best_estimator = #TODO
best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

lr_b = float(best_estimator.hyperparameters()["lr"])
epochs_b = int(best_estimator.hyperparameters()['epochs'])
#batch_size_b = int(best_estimator.hyperparameters()['batch_size']
batch_size_b = int(best_estimator.hyperparameters()['batch_size'].replace('"', ""))

print("Best hyperparameters from hpo:")
print("lr:",lr_b)
print("batch_size:",batch_size_b)
print("epochs:",epochs_b)

In [27]:
#Best hyperparameters from hpo to avoid re-running previous jobs
lr_b = 0.09223091926609225
batch_size_b = 257
epochs_b = 4



hyperparameters_b = {
    "lr": lr_b,
    "batch_size": batch_size_b,
    "epochs": epochs_b
}

print("Best hyperparameters from hpo:")
print("lr:",lr_b)
print("batch_size:",batch_size_b)
print("epochs:",epochs_b)



Best hyperparameters from hpo:
lr: 0.09223091926609225
batch_size: 257
epochs: 4


## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [30]:
# TODO: Set up debugging and profiling rules and hooks

"""
#Define rules:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing(),
                   rule_parameters={"tensor_regex": "CrossEntropyLoss_output_0","mode": "TRAIN"}),#forum
    #ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    #Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    #Rule.sagemaker(rule_configs.poor_weight_initialization()),
]

#Define hooks
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)

debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)
"""

#a revisar y borrar
#collection_configs=[CollectionConfig(name="CrossEntropyLoss_output_0",parameters={
#    "include_regex": "CrossEntropyLoss_output_0", "train.save_interval": "10","eval.save_interval": "1"})]
#debugger_config=DebuggerHookConfig( collection_configs=collection_configs )



# TODO: Set up debugging and profiling rules and hooks
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [
    Rule.sagemaker(base_config=rule_configs.loss_not_decreasing(),
                     rule_parameters={"tensor_regex": "CrossEntropyLoss_output_0",
                                     "mode": "TRAIN"}),#forum
    #ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    #Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    #Rule.sagemaker(rule_configs.poor_weight_initialization()),
]

#Define the hooks
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)



In [ ]:
# TODO: Create and fit an estimator
path =  "s3://{}/dogImages".format(bucket)
#estimator = # TODO: Your estimator here
"""
estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    #instance_type="ml.p3.2xlarge",#this is too expensive
    instance_type="ml.c5.2xlarge",
    entry_point="train_model.py",
    framework_version="1.8",
    py_version="py36",
    hyperparameters=hyperparameters_b,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
    output_path=path
)
"""

estimator = sagemaker.pytorch.estimator.PyTorch(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    #instance_type="ml.p3.2xlarge",#this is too expensive
    #instance_type="ml.c5.2xlarge",
    instance_type="ml.m5.large",#much cheaper
    entry_point="train_model.py",
    framework_version="1.8",
    py_version="py36",
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    hyperparameters = hyperparameters_b,
    rules=rules,
    output_path=path
)


"""
estimator.fit({
'train': path + '/train',     
'valid': path + '/valid',  
'test': path + '/test'     
})
"""

estimator.fit({
'train': path + '/train',     
'test': path + '/test',      
}, wait=True)

#last job took 70 min as follows: #start: 8:53 end: 10:03. It is not necessary to run it again
#this job started at 19:07 (00:07UTC) and finished at 19:37, so it took about 30min
#started 9:53, finishing by 10:23
#started 10:25, by 10:50
#started 10:53, by 11:28

2022-01-13 15:53:44 Starting - Starting the training job...
2022-01-13 15:54:10 Starting - Launching requested ML instancesLossNotDecreasing: InProgress
VanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2022-01-13 15:55:11 Starting - Preparing the instances for training.........
2022-01-13 15:56:38 Downloading - Downloading input data.........
2022-01-13 15:58:12 Training - Downloading the training image.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-13 15:58:17,579 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-13 15:58:17,581 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-01-13 15:58:17,589 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-01-13 15:58:20

In [ ]:
pt_model_data = estimator.model_data
print("Model artifact saved at:\n", pt_model_data)
import boto3

session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

In [ ]:
!date

In [ ]:
# TODO: Plot a debugging output.
#Checking Training Performance
#From 4.8.SageMaker Debugger.rtfd
import smdebug
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

print("Tensor names:",trial.tensor_names())

In [ ]:
from smdebug import modes
trial.tensor('CrossEntropyLoss_output_0').values(mode=modes.TRAIN)

In [ ]:

"""
import matplotlib.pyplot as plt
plt.ylabel('Training Loss')
plt.xlabel('Steps')
plt.plot(trial.steps(mode=modes.TRAIN),
         list(trial.tensor('CrossEntropyLoss_output_0').values(mode=modes.TRAIN).values()))
plt.show
"""
;

In [ ]:
# TODO: Plot a debugging output.
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot

def get_data(trial, tname, mode):
    tensor = trial.tensor(tname)
    steps = tensor.steps(mode=mode)
    vals = []
    for s in steps:
        vals.append(tensor.value(s, mode=mode))
    return steps, vals



#finally the plot
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot

def plot_tensor(trial, tensor_name):
    steps_train, vals_train = get_data(trial, tensor_name, mode=ModeKeys.TRAIN)
    print("loaded TRAIN data")
    steps_eval, vals_eval = get_data(trial, tensor_name, mode=ModeKeys.EVAL)
    print("loaded EVAL data")

    fig = plt.figure(figsize=(10, 7))
    host = host_subplot(111)

    par = host.twiny()

    host.set_xlabel("Steps (TRAIN)")
    par.set_xlabel("Steps (EVAL)")
    host.set_ylabel(tensor_name)

    (p1,) = host.plot(steps_train, vals_train, label=tensor_name)
    print("completed TRAIN plot")
    (p2,) = par.plot(steps_eval, vals_eval, label="val_" + tensor_name)
    print("completed EVAL plot")
    leg = plt.legend()

    host.xaxis.get_label().set_color(p1.get_color())
    leg.texts[0].set_color(p1.get_color())

    par.xaxis.get_label().set_color(p2.get_color())
    leg.texts[1].set_color(p2.get_color())

    plt.ylabel(tensor_name)

    plt.show()
    


#plot_tensor(trial, "nll_loss_output_0")
plot_tensor(trial,'CrossEntropyLoss_output_0')

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it? 


The graph shows no sustained tendency for different the batch sets. Which is understable since there is 

**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?


Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?

Batches could be shuffled to try to minimize this effect.



In [ ]:
# TODO: Display the profiler output

#From 4.9.SageMaker Profiler.rtfd


from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()




In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)



rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")



In [ ]:
! aws s3 ls {rule_output_path} --recursive

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive

In [ ]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [ ]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint
from sagemaker.model_monitor import DataCaptureConfig

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=64,
    destination_s3_uri=f"s3://{bucket}/data_capture",
)

# TODO: Add your deployment configuration like instance type and number of instances
predictor=estimator.deploy(instance_type="ml.m5.large", initial_instance_count=1) 


In [ ]:
# TODO: Run an prediction on the endpoint

#image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
#response = predictor.predict(image)
# TODO: Run an prediction on the endpoint
# TODO: Your code to load and preprocess image to send to endpoint for prediction
import torch
import torchvision
from torchvision import transforms

testing_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


testset = torchvision.datasets.ImageFolder(root="sample_image", 
            transform=testing_transform)
image = torch.utils.data.DataLoader(testset)
for input, label in image:
    print(predictor.predict(input))
                             

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()